# In this lesson, we will

- Study the `JsonHttpAction` Sample Manifest File
- Use the `requests` library to create a Pet against our local server (api-server-one)
- Create a JsonHttpAction Manifest File to create a Pet
- Run the framework and examine the Report

# Step 1: Examine the Template json_http_action.yaml

In [21]:
!cat ./api_client_one/manifests/actions/json_http_action.yaml

model_name: JsonHttpActionModel
config:
  method:
    format: string
    template: GET
  params:
    format: yaml
    template: |
      abc: abc
  url:
    format: string
    template: "http://www.abc.com"
description: Dummy Json Http Request
execution_id: 'json_http_action'
schemas:
  - id: "json_200"
    model_name: JsonSchemaModel
    schema_:
      format: json
      text: |
        {
          "$schema": "http://json-schema.org/draft-07/schema#",
          "type": "object",
          "additionalProperties": false
          }

schema_validators:
  - id: "json_200"
    model_name: JsonSchemaValidatorModel
    schema_id: "200"
    against: 'output_body'


# Explanation of JsonHttpAction Manifest

- *model_name*: A mandatory field which indicates which underlying model we are using. `JsonHttpActionModel` refers to a Model which communicates using JSON as body over HTTP. 

- *config*: This section contains all the user-input parameters relevant to the model. All config items are jinja-templateable. Examples are method (Http Method), params, URL, body and headers. All configurable items require `format` (as in what it should be converted to) and `template`.

- *description*: A description of what the Action is about

- *execution_id*: This ID must be unique in a ScenarioModel. A ScenarioModel contains a series of ActionModels. This will be discussed in another lesson. When left blank, the execution_id of an action manifest will take the same name as the name of the manifest file.

- *schemas*: A list of either JsonSchema or XmlSchema specific to this ActionModel. Can be an empty list.

- *schema_validators*: A list of mapping which maps any one schema in schemas (specified via the `schema_id` field) with the any field in the output field (auto-generated by the programme, specified via the `against` field)



# Step 2: Programmatically create a Pet entry using requests library

## Confirm that the Status code is `200`. It means that the pet is successfully created

In [22]:
# Create a Pet 
import requests
from datetime import datetime

pet_id = 1
url = f'http://localhost:8080/pets/{pet_id}'
body = {
  "animal_type": "cat",
  "name": "Susie",
  "tags": {}
}

resp = requests.put(url=url,json=body)

print(f"{resp.status_code=}")
print(f"{resp.text=}")
print(f"{resp.headers=}")

resp.status_code=200
resp.text=''
resp.headers={'Server': 'Werkzeug/2.2.3 Python/3.11.5', 'Date': 'Sun, 24 Sep 2023 07:54:03 GMT', 'Content-Type': 'application/json', 'Content-Length': '0', 'Connection': 'close'}


# Step 3: Create the JsonHttpAction Manifest for creating a Pet

## Note we will modify the below fields

| Field | Value | Reason |
| --- | --- | --- |
| method_name | JsonHttpActionModel | We send and receives Json as Body over Http |
| config.method | format is string, template is 'PUT' | API Server requests PUT method to create a pet |
| config.params | format is YAML, template is '' | No query parameters are added to the URL  |
| config.URL | format is string, template is 'http://localhost:8080/pets/1' | That is the URL (without the query parameters) |
| description | Create a Pet | Description describing what the Action does |
| execution_id | 'create_pet' | A unique name of the action in a Scenario. Let's name it the same as our file first, `create_pet.yaml` |
| schemas | [] | empty list for now.  |
| schema_validators | [] | empty list for now. |


In [23]:
%%writefile ./api_client_one/manifests/actions/create_pet.yaml
model_name: JsonHttpActionModel
config:
  method:
    format: string
    template: PUT
  params:
    format: yaml
    template: ''
  url:
    format: string
    template: "http://localhost:8080/pets/1"
description: Dummy Json Http Request
execution_id: 'json_http_action'

Overwriting ./api_client_one/manifests/actions/create_pet.yaml


In [24]:
# Check that the file is written

! cat ./api_client_one/manifests/actions/create_pet.yaml

model_name: JsonHttpActionModel
config:
  method:
    format: string
    template: PUT
  params:
    format: yaml
    template: ''
  url:
    format: string
    template: "http://localhost:8080/pets/1"
description: Dummy Json Http Request
execution_id: 'json_http_action'


# Step 4: Execute the Action Manifest

In [25]:
!bash -c "cd ./api_client_one && acp run -f ./manifests/actions/create_pet.yaml"

/home/ken/PycharmProjects/api_compose/venv/lib/python3.11/site-packages/pydantic/_internal/_utils.py:319: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not obj and obj_type in BUILTIN_COLLECTIONS:
[EventType.Deserialisation] : INFO - Parsed CLI context
[EventType.Deserialisation] : INFO - {}
[EventType.Default] : INFO - Overriding selectors pack to `custom`
[EventType.Discovery] : DEBUG - Display Current Env Files Pack: base
[EventType.Discovery] : DEBUG - Display Environment Variables:
[EventType.Discovery] : DEBUG - {
    "globals": {},
    "actions": {},
    "scenarios": {}
}
[EventType.Discovery] : DEBUG - Display Current Selector Pack: custom
[EventType.Session] : INFO - Running Session session_model.id='5845f967-ed82-4eba-b0e0-8a3055547ce5'
5845f967-ed82-4eba-b0e0-8a3055547ce5.spec_create_pet
[EventType.Specification] : INFO - Running Specification spec_create_pet
[EventType.Scenario] :

# Step 5: Read the HTML Report

In [26]:
from IPython.display import HTML

HTML('build/reports/.52b59085-61cc-4347-9d49-851592d6cf51/SessionModel_report.html')